<a href="https://colab.research.google.com/github/Sharathrao98/osds/blob/master/SampleOSDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install osds-channel-preview

     |████████████████████████████████| 92kB 5.7MB/s 
  Created wheel for osds-channel-preview: filename=osds_channel_preview-1.0.41-cp36-none-any.whl size=6014 sha256=618644c117b6d1cba2ec1e611e59d42fbe47000bc589ef6203973554f73acc60
  Stored in directory: /root/.cache/pip/wheels/49/bf/cd/49cc055316e028afe328314ec61765fded32a21c856a76bdeb
Successfully built osds-channel-preview


In [ ]:
from osds.utils import ObjectStorageDataset
from torch.utils.data import DataLoader

In [ ]:
train_ds = ObjectStorageDataset(f"gcs://gs://cloud-training-demos/taxifare/large/taxi-train*.csv",  
                                       storage_options = {'anon' : False }, 
                                       dtype='float32',
                                       batch_size = 16384, 
                                       eager_load_batches=False)

train_dl = DataLoader(train_ds, batch_size=None)

In [ ]:
import torch as pt
LEARNING_RATE = 0.03

device = "cuda:0" if pt.cuda.is_available() else "cpu"
print(device)
model = pt.nn.Linear(4, 1).to(device)

optimizer = pt.optim.SGD(model.parameters(), lr=LEARNING_RATE)

cuda:0


In [ ]:
GRADIENT_NORM = 0.1

ITERATION_COUNT = 10000

losses = []
for iter_idx, batch in zip(range(ITERATION_COUNT), train_dl):
  y_batch, X_batch = batch[:, 0].to(device), batch[:, 3:].to(device)

  y_est = model(X_batch)
  mse = pt.mean( (y_est - y_batch) ** 2 )
  mse.backward()

  pt.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_NORM) if GRADIENT_NORM else None

  optimizer.step()
  optimizer.zero_grad()

  losses.append(mse.data.item())
  if (iter_idx % 100 == 0):
    print(f"Iteration: {iter_idx}, MSE: {mse.data.item()}, W: {model.weight.data.squeeze()}")

Iteration: 0, MSE: 80.60306549072266, W: tensor([ 0.2264, -0.1425, -0.1535,  0.5529], device='cuda:0')
Iteration: 100, MSE: 74.62460327148438, W: tensor([ 0.2320, -0.1456, -0.1479,  0.5498], device='cuda:0')
Iteration: 200, MSE: 122.206787109375, W: tensor([ 0.2245, -0.1415, -0.1553,  0.5538], device='cuda:0')
Iteration: 300, MSE: 76.9235610961914, W: tensor([ 0.2297, -0.1444, -0.1502,  0.5509], device='cuda:0')
Iteration: 400, MSE: 73.5997085571289, W: tensor([ 0.2259, -0.1424, -0.1539,  0.5529], device='cuda:0')
Iteration: 500, MSE: 73.38966369628906, W: tensor([ 0.2296, -0.1444, -0.1502,  0.5508], device='cuda:0')
Iteration: 600, MSE: 119.76499938964844, W: tensor([ 0.2259, -0.1424, -0.1539,  0.5529], device='cuda:0')
Iteration: 700, MSE: 78.57455444335938, W: tensor([ 0.2296, -0.1444, -0.1502,  0.5508], device='cuda:0')
Iteration: 800, MSE: 61.352149963378906, W: tensor([ 0.2296, -0.1444, -0.1502,  0.5508], device='cuda:0')
Iteration: 900, MSE: 70.09423828125, W: tensor([ 0.2296, -